In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process

In [28]:
# Load dataset
df = pd.read_csv('C:/Users/BADAWY/Desktop/GP- Project/Data Set/Main_Final_with_user_data.csv', encoding='utf-8-sig')

In [29]:
import pandas as pd
import ftfy  # مكتبة إصلاح النصوص

# تحميل البيانات
file_path = 'C:/Users/BADAWY/Desktop/GP- Project/Data Set/Main_Final_with_user_data.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')  # حاول استخدام الترميز utf-8-sig

# دالة لإصلاح النصوص المشوهة باستخدام ftfy
def fix_garbled_text(text):
    return ftfy.fix_text(text)

# تطبيق الإصلاح على الأعمدة
for col in ['Title', 'Author', 'Category']:
    df[col] = df[col].apply(fix_garbled_text)

# طباعة عينة من البيانات بعد الإصلاح
print(df[['Title', 'Author']].sample(10))

# طباعة أول 5 صفوف من البيانات للتأكد من التصحيح
print(df.head())


                                                  Title  \
3251                          'Go to sleep, my darling'   
2435                                    'Poison Planet'   
3204                                    'Mating center'   
1829                                         'Goslings'   
2611                                   "Bratton's Idea"   
2123                                'World in a Mirror'   
2408                                       'Signal Red'   
176   'The Sleeper Awakes: A Revised Edition of When...   
1214                     'Queen of the Flaming Diamond'   
1230                                        'The Brain'   

                                    Author  
3251  'Marks, Winston K. (Winston Kinney)'  
2435                  'Oberfield, William'  
3204                 'Long, Frank Belknap'  
1829       'Beresford, J. D. (John Davys)'  
2611                 'Wellman, Manly Wade'  
2123                    'Teichner, Albert'  
2408                         'Guth,

In [30]:
print(df.head())

   ID                                          Title               Category  \
0  35                             'The Time Machine'        Science Fiction   
1  36                        'The War of the Worlds'        Science Fiction   
2  42  'The Strange Case of Dr. Jekyll and Mr. Hyde'  Horror/Gothic Fiction   
3  43  'The Strange Case of Dr. Jekyll and Mr. Hyde'  Horror/Gothic Fiction   
4  62                           'A Princess of Mars'        Science Fiction   

                            Author  \
0  'Wells, H. G. (Herbert George)'   
1  'Wells, H. G. (Herbert George)'   
2        'Stevenson, Robert Louis'   
3        'Stevenson, Robert Louis'   
4          'Burroughs, Edgar Rice'   

                                             Summary  \
0  A scientist invents a machine that allows him ...   
1  Earth is invaded by Martians with advanced tec...   
2  A respected scientist creates a potion that tr...   
3  A man is transported to Mars, where he discove...   
4  The sequel fo

In [31]:
# Clean up missing values in critical fields
df = df.dropna(subset=['Title', 'Category', 'Author'])  # Remove 'Description' if it's missing

# Combine multiple text features for better similarity (Title, Category, Author)
df['combined_features'] = df['Title'] + ' ' + df['Category'] + ' ' + df['Author']

In [32]:
# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

In [33]:
# Create the TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

In [34]:
print(tfidf_matrix.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [35]:
# Calculate the similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

In [36]:
print(similarity_matrix)

[[1.         0.49284068 0.00679351 ... 0.01152076 0.         0.00720069]
 [0.49284068 1.         0.0066062  ... 0.01120311 0.         0.00700215]
 [0.00679351 0.0066062  1.         ... 0.00620772 0.         0.00387994]
 ...
 [0.01152076 0.01120311 0.00620772 ... 1.         0.02878739 0.07113085]
 [0.         0.         0.         ... 0.02878739 1.         0.01799264]
 [0.00720069 0.00700215 0.00387994 ... 0.07113085 0.01799264 1.        ]]


In [37]:
# Fuzzy matching to find the closest book title match
def get_best_match(input_title, titles):
    titles = titles.str.strip().str.lower()
    input_title = input_title.strip().lower()
    match, score, _ = process.extractOne(input_title, titles)
    index = titles[titles == match].index[0] if match in titles.values else None
    return match, score, index

In [38]:
# Recommendation function
def recommend_books(book_title, data, similarity_matrix):
    # Get the best fuzzy match for the input title
    best_match, score, book_index = get_best_match(book_title, data['Title'])

    if book_index is None:
        print("🔍 No good match found for the given title.")
        return []

    # Get similarity scores for all books
    similar_books = list(enumerate(similarity_matrix[book_index]))

    # Exclude the same book and sort by similarity score
    similar_books = sorted(similar_books, key=lambda x: x[1], reverse=True)[1:50]

    # Build the list of recommendations
    recommendations = []
    for idx, sim_score in similar_books:
        recommendations.append({
            'title': data.iloc[idx]['Title'],
            'similarity': float(round(sim_score, 2)),
            'rating': int(data.iloc[idx].get('Rating', 0)),
            'cover': data.iloc[idx]['Cover']
        })

    # ✅ Sort by highest rating first, then by highest similarity
    recommendations = sorted(recommendations, key=lambda x: (x['rating'], x['similarity']), reverse=True)

    return recommendations

# Example usage and display
recommendations = recommend_books("The War of the Worlds", df, similarity_matrix)


In [41]:
from IPython.display import display, HTML

def display_recommendations_grid(recommendations):
    html_content = '<div style="display: flex; flex-wrap: wrap; gap: 20px;">'

    for rec in recommendations:
        html_content += f"""
        <div style="width: 180px; border: 1px solid #ddd; border-radius: 10px; padding: 10px; text-align: center;">
            <img src="{rec['cover']}" alt="cover" style="width: 100%; height: 250px; object-fit: cover; border-radius: 5px;">
            <h4 style="font-size: 16px; margin: 10px 0;">{rec['title']}</h4>
            <p style="margin: 5px 0;">⭐ <strong>{rec['rating']}</strong></p>
            <p style="margin: 0;">🔁 <strong>{rec['similarity']}</strong></p>
        </div>
        """

    html_content += '</div>'
    display(HTML(html_content))

# Example usage
recommendations = recommend_books("time of machine", df, similarity_matrix)
display_recommendations_grid(recommendations[:12])  # Show top 12 recommendations
